In [123]:
import requests
from shapely.geometry import Polygon, mapping
from datetime import datetime
import netrc
import getpass
from subprocess import PIPE, Popen
import os

### Credentials setup for EarthData

In [116]:
ASF_USER = input("Enter Username: ")
ASF_PASS = getpass.getpass("Enter Password: ")

Enter Username:  pbillecocq
Enter Password:  ···············


### Function definitions

In [143]:
def build_bbox_string(polygon):
    '''
    Builds the string to include in the ASF search request. The bbox consists of 4 comma-separated numbers: lower left longitude,latitude, and upper right longitude,latitude.
    
    Parmeters
    ----------
    polygon: shapely polygon
    
    Returns
    ----------
    polygon_string : string
        String to include in the ASF request
    '''
    points = mapping(polygon)['coordinates'][0]
    lower_left = points[0]
    upper_right = points[2]
    bbox_string = f'{lower_left[0]},{lower_left[1]},{upper_right[0]},{upper_right[1]}'
        
    return bbox_string

def search_asf(platform, processingLevel, start, end, polygon, output_format):
    '''
    Search the ASF platform for images given the input parameters
    
    Parameters
    ----------
        platform : string
            Name of the imaging platform. Defaults to UAVSAR, but a list of supported platform is available on the ASF website
        processingLevel : string
            Processing level of the imaging product. 
            Possible values for UAVSAR : (KMZ, PROJECTED, PAULI, PROJECTED_ML5X5, STOKES, AMPLITUDE, BROWSE, COMPLEX, DEM_TIFF, PROJECTED_ML3X3, METADATA, AMPLITUDE_GRD, INTERFEROMETRY, INTERFEROMETRY_GRD, THUMBNAIL)
        start : datetime object
            Start date of the search period.
        end : datetime object
            End date of the search period.
        polygon : shapely polygon defining the Area of Interest,
        output_format: string
            Format being returned by the ASF API. Values : CSV, JSON, KML, METALINK, COUNT, DOWNLOAD, GEOJSON
        
    Returns
    -------
    Ouputs a search file
    '''
    base = 'https://api.daac.asf.alaska.edu/services/search/param'
    start_date = start.strftime('%Y-%m-%dT%H:%M:%SUTC')
    end_date = end.strftime('%Y-%m-%dT%H:%M:%SUTC')
    aoi_string = build_bbox_string(polygon)
    payload = {
        'platform': platform,
        'processingLevel': processingLevel,
        'start': start,
        'end': end,
        'bbox': aoi_string,
        'output': output_format
    }
    #request_string = build_asf_search_request(platform, processingLevel, start, end, polygon)
    # Perform actual request to the asf
    r = requests.get(base, params=payload)
    
    return r.json()

def download_single_product(productUrl, path):
    '''
    Downloading function.
    
    Paramters
    ----------
    productUrl: string
            Product download Url
    path: string
            path to download product to
            
    Returns
    -------
    Download the file
    '''
    process = Popen(['wget', productUrl, f'--user={ASF_USER}', f'--password={ASF_PASS}', '-P', path, '--progress=bar'], stderr=PIPE)
    started = False
    for line in process.stderr:
        line = line.decode("utf-8", "replace")
        if started:
            splited = line.split()
            if len(splited) == 9:
                percentage = splited[6]
                speed = splited[7]
                remaining = splited[8]
                print("Downloaded {} with {} per second and {} left.".format(percentage, speed, remaining), end='\r')
        elif line == os.linesep:
            started = True
    print(f"\nDone downloading {productUrl}")
    
def search_and_bulk_download(platform, processingLevel, start, end, polygon, path):
    '''
    Search the ASF platform for images given the input parameters. Download found products
    
    Parameters
    ----------
        platform : string
            Name of the imaging platform. Defaults to UAVSAR, but a list of supported platform is available on the ASF website
        processingLevel : string
            Processing level of the imaging product. 
            Possible values for UAVSAR : (KMZ, PROJECTED, PAULI, PROJECTED_ML5X5, STOKES, AMPLITUDE, BROWSE, COMPLEX, DEM_TIFF, PROJECTED_ML3X3, METADATA, AMPLITUDE_GRD, INTERFEROMETRY, INTERFEROMETRY_GRD, THUMBNAIL)
        start : datetime object
            Start date of the search period.
        end : datetime object
            End date of the search period.
        polygon : shapely polygon defining the Area of Interest,
        path: string
            Path to download product to
        '''
    results = search_asf(platform, processingLevel, start, end, polygon, output_format='JSON')[0]
    print(f'{len(results)} product(s) found')
    for result in results:
        downloadUrl = result['downloadUrl']
        download_product(downloadUrl, path)
        print('Unzipping product')
        with zipfile.ZipFile(path + result['fileName'], 'r') as zip_ref:
            zip_ref.extractall(path + result['productName'])
        

### Testing playground

In [138]:
# Gran Mesa polygon for testing purposes
gran_mesa_polygon = Polygon([(-108.2883, 38.882), (-108.0001, 38.882), (-108.0001, 39.0256), (-108.2883, 39.0256), (-108.2883, 38.882)])
start_date = datetime.strptime('2020-01-10 11:00:00', '%Y-%m-%d %H:%M:%S') 
end_date = datetime.strptime('2020-02-10 11:00:00', '%Y-%m-%d %H:%M:%S') 

In [84]:
result = search_asf(platform='UAVSAR', processingLevel='INTERFEROMETRY_GRD', start=start_date, end=end_date, polygon=gran_mesa_polygon, output_format='JSON')

In [144]:
search_and_bulk_download(platform='UAVSAR', processingLevel='INTERFEROMETRY_GRD', start=start_date, end=end_date, polygon=gran_mesa_polygon, path='./')

4 product(s) found
Downloaded 99% with 335M per second and 0s left....
Done 
Downloaded 99% with 33.7M per second and 0s left...
Done 
Downloaded 99% with 3.55M per second and 0s left...
Done 
Downloaded 99% with 24.3M per second and 0s left....
Done 


In [134]:
download_product(file, data_dir)

Downloaded 99% with 15.6M per second and 0s left...
Done 
